In [1]:
import numpy as np
import tensorflow as tf
import gym
import os

/home/armughan/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/armughan/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def form_placeholders(input_shape):
    X=tf.placeholder(dtype=tf.float32,shape=[None,input_shape])
    return X

In [3]:
def form_hidden_layers(inputs,num_neurons_in_all_layers,activation_fn,initializer):
    layer_inputs=inputs
    for num_neurons in num_neurons_in_all_layers:
        layer_outputs=tf.layers.dense(layer_inputs,num_neurons,activation=activation_fn,kernel_initializer=initializer)
        layer_inputs=layer_outputs
    return layer_outputs

In [4]:
def form_loss(logits,targets,loss_fn):
    entropies=loss_fn(labels=targets,logits=logits)
#     loss=tf.reduce_mean(entropies)
#     return loss
    return entropies

In [5]:
def get_gradients_and_optimizer(loss,learning_rate,optimizer_fn):
    optimizer=optimizer_fn(learning_rate)
    grads_and_vars=optimizer.compute_gradients(loss)
    return optimizer,grads_and_vars

In [6]:
def update_weights(optimizer,grads_and_vars):
    grad_placeholders_list=[]
    grads_and_vars_feed=[]
    for grad,var in grads_and_vars:
        grad_placeholder=tf.placeholder(tf.float32,shape=grad.get_shape())
        grad_placeholders_list.append(grad_placeholder)
        grads_and_vars_feed.append((grad_placeholder,var))
    train_op=optimizer.apply_gradients(grads_and_vars_feed)
    return grad_placeholders_list,grads_and_vars_feed,train_op

In [17]:
class ANN:
    def __init__(self,params):
        input_shape=params['input_shape']
        target_shape=params['target_shape']
        num_outputs=params['num_outputs']
        num_neurons_in_hidden_layers=params['num_neurons']
        activation_fn=params.get('activation_fn',tf.nn.relu)
        loss_fn=params['loss_fn']
        learning_rate=params['learning_rate']
        optimizer_fn=params['optimizer_fn']
        logdir=params['logdir']
        
        tf.reset_default_graph()
        initializer_fn=tf.contrib.layers.variance_scaling_initializer()
        self.X=form_placeholders(input_shape)
        last_hidden_output=form_hidden_layers(self.X,num_neurons_in_hidden_layers,activation_fn,initializer_fn)
        
        logits=tf.layers.dense(last_hidden_output,num_outputs,kernel_initializer=initializer_fn)
        self.logits=logits
        self.outputs=tf.nn.sigmoid(logits)
        p_left_and_right=tf.concat(axis=1,values=[self.outputs,1-self.outputs])
        self.action=tf.multinomial(tf.log(p_left_and_right),num_samples=1)
        
        targets=1-tf.to_float(self.action)
        
        self.entropies=form_loss(logits,targets,loss_fn)
        self.optimizer,grads_and_vars=get_gradients_and_optimizer(self.entropies,learning_rate,optimizer_fn)
        self.gradients=[grad for grad,variable in grads_and_vars]
        
        
        
        self.grad_placeholders_list=[]
        self.grads_and_vars_feed=[]
        for grad,var in grads_and_vars:
            grad_placeholder=tf.placeholder(tf.float32,shape=grad.get_shape())
            self.grad_placeholders_list.append(grad_placeholder)
            self.grads_and_vars_feed.append((grad_placeholder,var))
        self.train_op=self.optimizer.apply_gradients(self.grads_and_vars_feed)
        
        
        
#         self.grad_placeholders_list,self.grads_and_vars_feed,self.train_op=update_weights(self.optimizer,grads_and_vars)
        
        self.initializer=tf.global_variables_initializer()
        self.saver=tf.train.Saver()
#         summ=tf.summary.scalar(self.entropies)
#         self.summaries=tf.summary.merge_all()
#         self.file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

In [18]:
def get_cumulative_discounted_rewards(rewards,discount_rate):#cumulates rewards for a single episode/game
    disc_rewards=np.empty(len(rewards))
    cum_rewards=0
    for step in reversed(range(len(rewards))):
        cum_rewards=rewards[step]+cum_rewards*discount_rate
        disc_rewards[step]=cum_rewards
    return disc_rewards  #returning dicounted rewards(same shape as the rewards in the parameters )
        

In [19]:
def normalize_and_cumulate_rewards(all_rewards,discount_rate):#cumulates and normalizes rewards over many episodes/games
    
    all_discounted_rewards=[get_cumulative_discounted_rewards(episode_reward,discount_rate) for episode_reward in all_rewards]
    flat_rewards=np.concatenate(all_discounted_rewards)
    reward_mean=flat_rewards.mean()
    reward_std=flat_rewards.std()
    return [(discounted_episode_rewards-reward_mean)/float(reward_std) for discounted_episode_rewards in all_discounted_rewards]

In [20]:
params={
    'input_shape':4,
    'target_shape':4,
    'num_outputs':1,
    'num_neurons':[4],
    'activation_fn':tf.nn.elu,
    'loss_fn':tf.nn.sigmoid_cross_entropy_with_logits,
    'learning_rate':0.01,
    'optimizer_fn':tf.train.AdamOptimizer,
    'logdir':'/tf_logs_rnn/run/'
}

In [21]:
# model=ANN(params)
# env=gym.make("CartPole-v0")
# n_iter=250
# n_games_per_iter=10
# n_steps_per_game=1000
# discount_rate=0.95
# savedir='/tmp/my_model_final.ckpt'
# with tf.Session() as sess:
# #     if os.path.isfile(savedir):
#     print ('restoring model')
#     model.saver.restore(sess,savedir)
# #     else:
# #     model.initializer.run()
#     for i in range(n_iter):
#         all_rewards=[]
#         all_gradients=[]#will contain grads of diff vars for many steps of many games/episodes so size=vars*steps_in_game*num_games
#         print ('current iteration '+str(i))
        
#         mean_entr=[]
        
#         for j in range(n_games_per_iter):
#             current_episode_rewards=[]
#             current_episode_grads=[]#will contain grads of diff variable for many steps of the current game so size=vars*no_of steps in this episode 
            
#             obs=env.reset()
#             for k in range(n_steps_per_game):
#                 feed_dict={model.X:obs.reshape(1,params['input_shape'])}
#                 action,grads,entropies=sess.run([model.action,model.gradients,model.entropies],feed_dict=feed_dict)
# #                 print (entropies)
#                 mean_entr.append(entropies)
                
#                 obs,reward,done,info=env.step(action[0][0])
#                 current_episode_rewards.append(reward)
#                 current_episode_grads.append(grads)
#                 if done:
#                     break
#             all_rewards.append(current_episode_rewards)
#             all_gradients.append(current_episode_grads)
            
#         #now we need to train/update model since the model has played   the game for n_games_per_iter 
#         all_normalized_rewards=normalize_and_cumulate_rewards(all_rewards,discount_rate) #normalizing the rewards so that the chances of a good action getting a bad score are reduced
#         feed_dict={}#for supplying model with the mean gradients multiplied by the rewards 
#         for var_index,grad_placeholder in enumerate(model.grad_placeholders_list):
#             mean_grads=np.mean(
#             [reward*all_gradients[game_index][step][var_index] for game_index,game_rewards in enumerate(all_rewards) for step,reward in enumerate(game_rewards)]
#             ,axis=0)
#             feed_dict[grad_placeholder]=mean_grads
#         sess.run([model.train_op],feed_dict=feed_dict)
#         model.saver.save(sess,savedir)
#         print ('entropy =')
#         print (np.mean(np.array(mean_entr)))
        
        
#     n_steps=1000
#     obs=env.reset()
#     for i in range(n_steps):
#         feed_dict={model.X:obs.reshape(1,params['input_shape'])}
#         outputs=sess.run(model.outputs,feed_dict=feed_dict)
#         action=0 if outputs>=0.5 else 1
#         obs,reward,done,info=env.step(action)
#         env.render()
#         print ('step no '+str(i))
#         if done:
#             print ('lost')
#             break   
# #     model.file_writer.close()

In [22]:
#testing phase
model=ANN(params)
env=gym.make("CartPole-v0")
n_iter=250
n_games_per_iter=10
n_steps_per_game=1000
discount_rate=0.95

save_iterations=10# save the model every 10 training iterations


with tf.Session() as sess:
    model.initializer.run()
    for iteration in range(n_iter):
        all_rewards=[]# all sequences of raw rewards for each episode
        all_gradients=[]# gradients saved at each step of each episode
        mean_entr=[]
        print ('current iteration '+str(iteration))
        for game in range(n_games_per_iter):
            current_rewards=[]# all raw rewards from the current episode
            current_gradients=[]# all gradients from the current episode
            obs=env.reset()
            for step in range(n_steps_per_game):
                action_val,gradients_val,entropies=sess.run(
                [model.action,model.gradients,model.entropies],
                feed_dict={model.X:obs.reshape(1,params['input_shape'])})# one obs
                obs,reward,done,info=env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                mean_entr.append(entropies)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)

        # At this point we have run the policy for 10 episodes, and we are
        # ready for a policy update using the algorithm described earlier.
        all_rewards=normalize_and_cumulate_rewards(all_rewards,discount_rate)
        feed_dict={}
        for var_index,grad_placeholder in enumerate(model.grad_placeholders_list):
            # multiply the gradients by the action scores, and compute the mean
            mean_gradients=np.mean(
            [reward*all_gradients[game_index][step][var_index]
            for game_index,rewards in enumerate(all_rewards)
            for step,reward in enumerate(rewards)],
            axis=0)
            feed_dict[grad_placeholder]=mean_gradients
        sess.run(model.train_op,feed_dict=feed_dict)
        if iteration%save_iterations==0:
            model.saver.save(sess,"./my_policy_net_pg.ckpt")
        print ('entropy =')
        print (np.mean(np.array(mean_entr)))
        
    n_steps=1000
    obs=env.reset()
    for i in range(n_steps):
        feed_dict={model.X:obs.reshape(1,params['input_shape'])}
        outputs_r=sess.run(model.outputs,feed_dict=feed_dict)
        action=0 if outputs_r>=0.5 else 1
        obs,reward,done,info=env.step(action)
        env.render()
        print ('step no '+str(i))
        if done:
            print ('lost')
            break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
current iteration 0
entropy =
0.6507775
current iteration 1
entropy =
0.65463793
current iteration 2
entropy =
0.65546
current iteration 3
entropy =
0.6974031
current iteration 4
entropy =
0.66370595
current iteration 5
entropy =
0.69489026
current iteration 6
entropy =
0.64200616
current iteration 7
entropy =
0.67697936
current iteration 8
entropy =
0.648913
current iteration 9
entropy =
0.6852597
current iteration 10
entropy =
0.68037456
current iteration 11
entropy =
0.6854181
current iteration 12
entropy =
0.6367241
current iteration 13
entropy =
0.68210673
current iteration 14
entropy =
0.68074423
current iteration 15
entropy =
0.688564
current iteration 16
entropy =
0.6774279
current iteration 17
entropy =
0.67497677
current iteration 18
entropy =
0.66992277
current iteration 19
entropy =
0.68276423
current iteration 20
entropy =
0.68212676
current iteration 21
entropy =
0.6930197
c

entropy =
0.5604634
current iteration 195
entropy =
0.56017524
current iteration 196
entropy =
0.5621141
current iteration 197
entropy =
0.55160683
current iteration 198
entropy =
0.5657002
current iteration 199
entropy =
0.55812114
current iteration 200
entropy =
0.55764073
current iteration 201
entropy =
0.5613217
current iteration 202
entropy =
0.5773436
current iteration 203
entropy =
0.55674505
current iteration 204
entropy =
0.5622384
current iteration 205
entropy =
0.5667107
current iteration 206
entropy =
0.5589674
current iteration 207
entropy =
0.5662775
current iteration 208
entropy =
0.56300884
current iteration 209
entropy =
0.55630255
current iteration 210
entropy =
0.56875616
current iteration 211
entropy =
0.5704565
current iteration 212
entropy =
0.56583685
current iteration 213
entropy =
0.5670971
current iteration 214
entropy =
0.56933767
current iteration 215
entropy =
0.5558031
current iteration 216
entropy =
0.5540253
current iteration 217
entropy =
0.56090236
cur